In [20]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage
import pandas as pd

In [3]:
CREDS = "neural-hour-421815-d95e4bee266b.json"
credentials = service_account.Credentials.from_service_account_file(CREDS)
client = bigquery.Client(credentials=credentials)
job_config = bigquery.QueryJobConfig()
        

In [14]:
df_file_path = "ETL/01_Data_Collection/01_dataset/processed_data/filtered_df3.csv"
df = pd.read_csv(df_file_path)
df['block'] = df['block'].astype(str)



/var/folders/wx/4zj8vyds7kv6w4t2k82r4lcr0000gn/T/ipykernel_56835/1184840066.py:2 DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

In [15]:
property_id = ['flat_type', 'flat_model', 'flat_type_model',
                  'floor_area_sqm', 'lease_commence_date', 'storey_range',
                  'avg_storey_range', 'total_dwelling_units']
        
address_id = ['postal','address', 'town', 'block', 'street_name', 
                                'planning_area', 'latitude', 'longitude']
        
surroundings_id = ['commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion',
                        'mall_nearest_distance', 'hawker_nearest_distance',
                                    'hawker_food_stalls', 'hawker_market_stalls', 'mrt_nearest_distance', 'mrt_name', 'bus_interchange', 'mrt_interchange', 'bus_stop_nearest_distance', 'bus_stop_name',
                                    'pri_sch_nearest_distance', 'pri_sch_name', 'vacancy', 'pri_sch_affiliation',
                                    'sec_sch_nearest_dist', 'sec_sch_name', 'cutoff_point', 'affiliation', 'school_type',
                                    'bus_stop_proximity', 'mrt_proximity', 'pri_sch_proximity', 'sec_sch_proximity']
transaction_id = ['month', 'year', 'quarter', 'inflation', 'normalized_resale_price']


property_df = df[property_id].copy().drop_duplicates()
property_df["property_id"] = [i for i in range(1 , len(property_df) + 1)]

address_df = df[address_id].copy().drop_duplicates()
address_df["address_id"] = [i for i in range(1 , len(address_df) + 1)]

surroundings_df = df[surroundings_id].copy().drop_duplicates()
surroundings_df["surroundings_id"] = [i for i in range(1 , len(surroundings_df) + 1)]

transaction_df = df[transaction_id].copy().drop_duplicates()
transaction_df["transaction_id"] = [i for i in range(1 , len(transaction_df) + 1)]


In [18]:
resale_data = df.copy()
resale_data["property_id"] = resale_data[property_id].merge(property_df, how='left')['property_id']
resale_data["address_id"] = resale_data[address_id].merge(address_df, how='left')['address_id']
resale_data["surroundings_id"] = resale_data[surroundings_id].merge(surroundings_df, how='left')['surroundings_id']
resale_data["transaction_id"] = resale_data[transaction_id].merge(transaction_df, how='left')['transaction_id']

# Drop the original columns that have been replaced
resale_data.drop(property_id + address_id + surroundings_id + transaction_id, axis=1, inplace=True)

In [26]:
project_id = "neural-hour-421815"
dataset_id = "is3107"  

In [30]:
credentials = service_account.Credentials.from_service_account_file(CREDS)

storage_client = storage.Client(credentials=credentials)
# Define the name of the GCS bucket and the local path to the pickle file
bucket_name = 'is3107_project'
# Get the specified bucket
bucket = storage_client.get_bucket(bucket_name)

    # List all objects (items) in the bucket
print(f"Objects in bucket '{bucket_name}':")
blobs = list(bucket.list_blobs())
if blobs:
    for blob in blobs:
        print(f"- {blob.name}")
else:
    print("No objects found in the bucket.")

Objects in bucket 'is3107_project':
- knn_model.pkl
- linear_model.pkl
- linear_model_new.pkl


In [16]:
table_id = "resale_price"   # Table ID within the specified dataset
df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")
        


  0%|          | 0/1 [00:00<?, ?it/s]

717069 out of 717069 rows loaded.m] {gbq.py:515} INFO - 


  0%|          | 0/1 [00:00<?, ?it/s]

717069 out of 717069 rows loaded.m] {gbq.py:515} INFO - 


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

8992 out of 8992 rows loaded.0] {gbq.py:515} INFO - 


  0%|          | 0/1 [00:00<?, ?it/s]

158174 out of 158174 rows loaded.m] {gbq.py:515} INFO - 


100%|██████████| 1/1 [00:00<00:00, 413.56it/s]


In [19]:
        # Upload DataFrame to BigQuery
table_id = "resale_data"   # Table ID within the specified dataset
resale_data.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")
        


  0%|          | 0/1 [00:00<?, ?it/s]

717069 out of 717069 rows loaded.m] {gbq.py:515} INFO - 


100%|██████████| 1/1 [00:00<00:00, 272.11it/s]


In [ ]:
        # Dataset ID and table name
table_id = "Property_new"   # Table ID within the specified dataset

        # Upload DataFrame to BigQuery
property_df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")
        
        # Dataset ID and table name
table_id = "Address_new"   # Table ID within the specified dataset

        # Upload DataFrame to BigQuery
address_df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")
        
        # Dataset ID and table name
table_id = "Surroundings_new"   # Table ID within the specified dataset

        # Upload DataFrame to BigQuery
surroundings_df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")
        
        # Dataset ID and table name
table_id = "Transaction_new"   # Table ID within the specified dataset

        # Upload DataFrame to BigQuery
transaction_df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")
